# Feature Extraction

In [18]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader 

In [33]:
# define image transformation
transform = transforms.Compose([
    transforms.Resize((352, 1216)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # why normalize?
])

In [111]:
# Set Directories
base_dir = 'dataset'
image_dir = os.path.join(base_dir, 'image')
depth_dir = os.path.join(base_dir, 'groundtruth_depth')
# Load Dataset
dataset = ImageFolder(base_dir, transform=transform)
dataloader = DataLoader(dataset, 32, False)
#! currently the dataset holds both the images and the groundtruths, while it should only hold the images !

In [48]:

def load_model(removed_layers = 1):
    # Load Pre-Trained Model
    model = models.resnet18(pretrained=True)
    #model = model.to("cpu")
    #remove final connected layers from model
    #model.fc = nn.Identity()
    model = torch.nn.Sequential(*list(model.children())[:-removed_layers])
    return model

In [103]:
# Function to extract features
def extract_features(dataloader, model = load_model(1)):
    features = []
    with torch.no_grad(): #Disables gradient calcullation
        for inputs, _ in dataloader:
            outputs = model(inputs)
            features.append(outputs.flatten(1))
    return torch.cat(features)

features = extract_features(dataloader, load_model(1))
features.shape

torch.Size([2000, 512])

# Reduce Dimensionality with LLE
1 512
2 214016
3 428032

In [23]:
import matplotlib.pyplot as plt
from sklearn.manifold import LocallyLinearEmbedding

In [32]:
#Hyperparamaters!
n_components = 150
n_neighbors = 25
lle = LocallyLinearEmbedding(n_components = n_components, n_neighbors = n_neighbors, method='standard')
features_lle = lle.fit_transform(features)
features.shape, features_lle.shape

(torch.Size([2000, 512]), (2000, 150))